In [1]:
import os
import obspy
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense
from pathlib import Path
%matplotlib inline
%run helperfunctions.ipynb
os.getcwd()

'/Users/albertkong/grillochallenge'

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
os.chdir('lse_challenge/data')

In [4]:
# Signal List
os.chdir('signal')
signals = []
signal_files = os.listdir()
signal_files = random.sample(signal_files, 750) #Need to expand for final model
for sigfile in signal_files: 
    tmp_trace = obspy.read(sigfile)
    tmp_trace = tmp_trace.normalize()
    cutlength = np.random.randint(16, 95) # Aligning with the [0.5 sec, 3 sec] range
    trace_mat = [tmp_trace[0].data[cutlength:1000+cutlength], tmp_trace[1].data[cutlength:1000+cutlength], tmp_trace[2].data[cutlength:1000+cutlength]]
    #mag = np.sqrt(trace_mat[0]**2 + trace_mat[1]**2 + trace_mat[2]**2)
    #signals.append([trace_mat[0], trace_mat[1], trace_mat[2], mag, cutlength, 1])
    signals.append([trace_mat[0], cutlength, 1])
    signals.append([trace_mat[1], cutlength, 1])
    signals.append([trace_mat[2], cutlength, 1])
    
    # Negative Transformations
    neg_mat = rot_xyz(trace_mat, 0, -np.pi, 0)
    #signals.append([neg_mat[0], neg_mat[1], neg_mat[2], mag, cutlength, 1])
    signals.append([neg_mat[0], cutlength, 1])
    signals.append([neg_mat[1], cutlength, 1])
    signals.append([neg_mat[2], cutlength, 1])
    
    # X, Y, and Z Rotations by Theta
    theta = np.random.uniform()*2*np.pi
    x_mat = rot_x(trace_mat, theta)
    #signals.append([x_mat[0], x_mat[1], x_mat[2], mag, cutlength, 1])
    signals.append([x_mat[0], cutlength, 1])
    signals.append([x_mat[1], cutlength, 1])
    signals.append([x_mat[2], cutlength, 1])
    y_mat = rot_y(trace_mat, theta)
    #signals.append([y_mat[0], y_mat[1], y_mat[2], mag, cutlength, 1])
    signals.append([y_mat[0], cutlength, 1])
    signals.append([y_mat[1], cutlength, 1])
    signals.append([y_mat[2], cutlength, 1])
    z_mat = rot_z(trace_mat, theta)
    #signals.append([z_mat[0], z_mat[1], z_mat[2], mag, cutlength, 1])
    signals.append([z_mat[0], cutlength, 1])
    signals.append([z_mat[1], cutlength, 1])
    signals.append([z_mat[2], cutlength, 1])    
    
    # General Rotation by alpha, beta, gamma
    alpha = np.random.uniform()*2*np.pi
    beta = np.random.uniform()*2*np.pi
    gamma = np.random.uniform()*2*np.pi
    rot_mat = rot_xyz(trace_mat, alpha, beta, gamma)
    #signals.append([rot_mat[0], rot_mat[1], rot_mat[2], mag, cutlength, 1])
    signals.append([rot_mat[0], cutlength, 1])
    signals.append([rot_mat[1], cutlength, 1])
    signals.append([rot_mat[2], cutlength, 1])
    

# Noise List
os.chdir('../noise')
noise_files = os.listdir()
noise_files = random.sample(noise_files, 2250) #Need to expand for final model
for noisefile in noise_files: 
    tmp_trace = obspy.read(noisefile)
    tmp_trace = tmp_trace.normalize()
    cutlength = np.random.randint(16, 95) # Aligning with the [0.5 sec, 3 sec] range
    trace_mat = [tmp_trace[0].data[cutlength:1000+cutlength], tmp_trace[1].data[cutlength:1000+cutlength], tmp_trace[2].data[cutlength:1000+cutlength]]
    #mag = np.sqrt(trace_mat[0]**2 + trace_mat[1]**2 + trace_mat[2]**2)
    #signals.append([trace_mat[0], trace_mat[1], trace_mat[2], mag, cutlength, 0])
    signals.append([trace_mat[0], cutlength, 0])
    signals.append([trace_mat[1], cutlength, 0])
    signals.append([trace_mat[2], cutlength, 0])
    
    # Negative Transformations
    neg_mat = rot_xyz(trace_mat, 0, -np.pi, 0)
    #signals.append([neg_mat[0], neg_mat[1], neg_mat[2], mag, cutlength, 0])
    signals.append([neg_mat[0], cutlength, 0])
    signals.append([neg_mat[1], cutlength, 0])
    signals.append([neg_mat[2], cutlength, 0])
    
    # X, Y, and Z Rotations by Theta
    theta = np.random.uniform()*2*np.pi
    x_mat = rot_x(trace_mat, theta)
    #signals.append([x_mat[0], x_mat[1], x_mat[2], mag, cutlength, 0])
    signals.append([x_mat[0], cutlength, 0])
    signals.append([x_mat[1], cutlength, 0])
    signals.append([x_mat[2], cutlength, 0])
    y_mat = rot_y(trace_mat, theta)
    #signals.append([y_mat[0], y_mat[1], y_mat[2], mag, cutlength, 0])
    signals.append([y_mat[0], cutlength, 0])
    signals.append([y_mat[1], cutlength, 0])
    signals.append([y_mat[2], cutlength, 0])
    z_mat = rot_z(trace_mat, theta)
    #signals.append([z_mat[0], z_mat[1], z_mat[2], mag, cutlength, 0])
    signals.append([z_mat[0], cutlength, 0])
    signals.append([z_mat[1], cutlength, 0])
    signals.append([z_mat[2], cutlength, 0])
    
    # General Rotation by alpha, beta, gamma
    alpha = np.random.uniform()*2*np.pi
    beta = np.random.uniform()*2*np.pi
    gamma = np.random.uniform()*2*np.pi
    rot_mat = rot_xyz(trace_mat, alpha, beta, gamma)
    #signals.append([rot_mat[0], rot_mat[1], rot_mat[2], mag, cutlength, 0])
    signals.append([rot_mat[0], cutlength, 0])
    signals.append([rot_mat[1], cutlength, 0])
    signals.append([rot_mat[2], cutlength, 0])
    
os.chdir('../')

sig_df = pd.DataFrame(signals, columns = ["trace", "cutlength", "signal"])
sig_df['p_arrival'] = 1000 - sig_df['cutlength']

## Training Neural Network for dectecting whether or not the trace has a signal

In [5]:
# For rolling averages
roll_short = 25
roll_long = 50

# For location segments
window_step = 30
window_size = 30

In [6]:
# Features and Targets for Identifying the Entire Trace

sigfeatures = []
sigtargets = []

for index, slice_df in sig_df.iterrows():
    # x = slice_df["tr_x"]
    # y = slice_df["tr_y"]
    # z = slice_df["tr_z"]
    tr = slice_df["trace"]
    mag = abs(tr)
    # mag = slice_df["magnitude"]
    signal = slice_df["signal"]
    p_arrival = slice_df["p_arrival"]
    #d = {"x": x, "y":y, "z":z, "magnitude":mag}
    d = {"trace": tr, "magnitude":mag}
    
    
    temp_df = pd.DataFrame(data = d)
    temp_df["STA"] = temp_df["magnitude"].rolling(roll_short).mean()
    temp_df["LTA"] = temp_df["magnitude"].rolling(roll_long).mean()
    temp_df["RAV"] = temp_df["STA"]/temp_df["LTA"]
    temp_df["STV"] = temp_df["magnitude"].rolling(roll_short).var()
    temp_df["LTV"] = temp_df["magnitude"].rolling(roll_long).var()
    
    temp_df.dropna(inplace = True)
    
    sigfeatures.append(temp_df.values)
    sigtargets.append(signal)

In [7]:
X_train_sig, X_test_sig, y_train_sig, y_test_sig = train_test_split(sigfeatures, sigtargets, test_size = 0.3)

In [8]:
X_train_sig = np.array(X_train_sig)
X_test_sig = np.array(X_test_sig)
y_train_sig = np.array(y_train_sig)
y_test_sig = np.array(y_test_sig)
feature_shape = X_train_sig[0].shape
feature_shape

(951, 7)

In [9]:
signalmodel = Sequential()
signalmodel.add(LSTM(32, input_shape = feature_shape))
signalmodel.add(Dense(1, activation = 'sigmoid'))
signalmodel.summary()

2022-03-10 01:23:42.863626: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-03-10 01:23:42.870612: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 32)                5120      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 5,153
Trainable params: 5,153
Non-trainable params: 0
_________________________________________________________________


In [10]:
signalmodel.compile(loss = 'binary_crossentropy', optimizer = "adam", metrics = ['accuracy'])
signalmodel.fit(X_train_sig, y_train_sig, epochs = 30)

Epoch 1/30


2022-03-10 01:23:48.934449: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-03-10 01:23:49.875963: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-10 01:23:50.196657: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-10 01:23:52.555604: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


788/788 [==============================] - 164s 196ms/step - loss: 0.1890 - accuracy: 0.9301
Epoch 2/30
788/788 [==============================] - 147s 186ms/step - loss: 0.0807 - accuracy: 0.9811
Epoch 3/30
788/788 [==============================] - 141s 179ms/step - loss: 0.1501 - accuracy: 0.9515
Epoch 4/30
788/788 [==============================] - 141s 179ms/step - loss: 0.0963 - accuracy: 0.9748
Epoch 5/30
788/788 [==============================] - 141s 179ms/step - loss: 0.1056 - accuracy: 0.9714
Epoch 6/30
788/788 [==============================] - 141s 179ms/step - loss: 0.0770 - accuracy: 0.9847
Epoch 7/30
788/788 [==============================] - 141s 179ms/step - loss: 0.0794 - accuracy: 0.9826
Epoch 8/30
788/788 [==============================] - 142s 180ms/step - loss: 0.1089 - accuracy: 0.9615
Epoch 9/30
788/788 [==============================] - 142s 180ms/step - loss: 0.3201 - accuracy: 0.8680
Epoch 10/30
788/788 [==============================] - 142s 180ms/step - lo

In [11]:
model_loss_sig, model_accuracy_sig = signalmodel.evaluate(X_test_sig, y_test_sig, verbose = 0)
print(f"Loss: {model_loss_sig}, Accuracy: {model_accuracy_sig}")

2022-03-10 02:36:11.207778: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-10 02:36:11.296201: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


Loss: 0.03033197671175003, Accuracy: 0.9912037253379822


In [12]:
predictions_test_sig = signalmodel.predict(X_test_sig)
predictions_train_sig = signalmodel.predict(X_train_sig)

2022-03-10 02:36:56.146847: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-10 02:36:56.182435: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [13]:
class_test_sig = predictions_test_sig > 0.2
f1_score_test_sig = sklearn.metrics.f1_score(y_test_sig, class_test_sig)
print(f"F1 Score: {f1_score_test_sig}")

F1 Score: 0.9841934869548657


In [14]:
class_train_sig = predictions_train_sig > 0.2
f1_score_train_sig = sklearn.metrics.f1_score(y_train_sig, class_train_sig)
print(f"F1 Score: {f1_score_train_sig}")

F1 Score: 0.9810967036400564


In [15]:
tn, fp, fn, tp = sklearn.metrics.confusion_matrix(y_test_sig, class_test_sig).ravel()
(tn, fp, fn, tp)

(8133, 48, 35, 2584)

In [16]:
# Exporting Model
os.getcwd()

'/Users/albertkong/grillochallenge/lse_challenge/data'

In [17]:
# Only run if we need to get to root directory
os.chdir('../../')

In [18]:
# Exporting signal model
signalmodel.save('signalmodel')

2022-03-10 02:39:16.557449: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: signalmodel/assets


INFO:tensorflow:Assets written to: signalmodel/assets


## Training Neural Network for dectecting where in the trace the p-wave starts

In [19]:
# Features and Targets for Identifying Windows of Trace
locfeatures = []
loctargets = []

for index, slice_df in sig_df.iterrows():
    
    signal = slice_df["signal"]
    
    if signal == 1:
        # x = slice_df["tr_x"]
        # y = slice_df["tr_y"]
        # z = slice_df["tr_z"]
        tr = slice_df["trace"]
        mag = abs(tr)
        # mag = slice_df["magnitude"]

        p_arrival = slice_df["p_arrival"]
        d = {"trace":tr, "magnitude":mag}

        temp_df = pd.DataFrame(data = d)
        temp_df["STA"] = temp_df["magnitude"].rolling(roll_short).mean()
        temp_df["LTA"] = temp_df["magnitude"].rolling(roll_long).mean()
        temp_df["RAV"] = temp_df["STA"]/temp_df["LTA"]
        temp_df["STV"] = temp_df["magnitude"].rolling(roll_short).var()
        temp_df["LTV"] = temp_df["magnitude"].rolling(roll_long).var()

        temp_df.dropna(inplace = True)

        start_ind = 0
        end_ind = start_ind + window_size



        while end_ind < (1000 - roll_long):
            # xwindow = temp_df["x"].iloc[start_ind:end_ind]
            # ywindow = temp_df["y"].iloc[start_ind:end_ind]
            # zwindow = temp_df["z"].iloc[start_ind:end_ind]
            trwindow = temp_df["trace"].iloc[start_ind:end_ind]
            magwindow = temp_df["magnitude"].iloc[start_ind:end_ind]
            # stawindow = temp_df["STA"].iloc[start_ind:end_ind]
            # ltawindow = temp_df["LTA"].iloc[start_ind:end_ind]
            ravwindow = temp_df["RAV"].iloc[start_ind:end_ind]
            stvwindow = temp_df["STV"].iloc[start_ind:end_ind]
            ltvwindow = temp_df["LTV"].iloc[start_ind:end_ind]

            window_data = {"trace": trwindow, "magnitude": magwindow,
                        "RAV": ravwindow, "STV": stvwindow, "LTV": ltvwindow}
            window_df = pd.DataFrame(data = window_data)

            locfeatures.append(window_df.values)

            if ((p_arrival-roll_long) >= start_ind) and ((p_arrival-roll_long) <= end_ind):
                loctargets.append(1)
            else:
                loctargets.append(0)

            start_ind += window_step
            end_ind = start_ind + window_size

In [20]:
# For Windows of Trace
X_train_loc, X_test_loc, y_train_loc, y_test_loc = train_test_split(locfeatures, loctargets, test_size = 0.3)

In [21]:
X_train_loc = np.array(X_train_loc)
X_test_loc = np.array(X_test_loc)
y_train_loc = np.array(y_train_loc)
y_test_loc = np.array(y_test_loc)
feature_shape = X_train_loc[0].shape
feature_shape

(30, 5)

In [22]:
locmodel = Sequential()
locmodel.add(LSTM(32, input_shape = feature_shape))
locmodel.add(Dense(1, activation = 'sigmoid'))
locmodel.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 32)                4864      
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 4,897
Trainable params: 4,897
Non-trainable params: 0
_________________________________________________________________


In [23]:
locmodel.compile(loss = 'binary_crossentropy', optimizer = "adam", metrics = ['accuracy'])
locmodel.fit(X_train_loc, y_train_loc, epochs = 15)

Epoch 1/15


2022-03-10 02:40:27.522054: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-10 02:40:27.687494: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


   5/6104 [..............................] - ETA: 1:34 - loss: 0.7623 - accuracy: 0.0500       

2022-03-10 02:40:27.801106: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


6104/6104 [==============================] - 79s 13ms/step - loss: 0.1078 - accuracy: 0.9704
Epoch 2/15
6104/6104 [==============================] - 78s 13ms/step - loss: 0.0957 - accuracy: 0.9747
Epoch 3/15
6104/6104 [==============================] - 78s 13ms/step - loss: 0.0917 - accuracy: 0.9759
Epoch 4/15
6104/6104 [==============================] - 79s 13ms/step - loss: 0.0890 - accuracy: 0.9765
Epoch 5/15
6104/6104 [==============================] - 79s 13ms/step - loss: 0.0876 - accuracy: 0.9770
Epoch 6/15
6104/6104 [==============================] - 78s 13ms/step - loss: 0.0862 - accuracy: 0.9775
Epoch 7/15
6104/6104 [==============================] - 78s 13ms/step - loss: 0.0852 - accuracy: 0.9779
Epoch 8/15
6104/6104 [==============================] - 78s 13ms/step - loss: 0.0841 - accuracy: 0.9781
Epoch 9/15
6104/6104 [==============================] - 78s 13ms/step - loss: 0.0834 - accuracy: 0.9785
Epoch 10/15
6104/6104 [==============================] - 78s 13ms/step - lo

In [24]:
model_loss_loc, model_accuracy_loc = locmodel.evaluate(X_test_loc, y_test_loc, verbose = 0)
print(f"Loss: {model_loss_loc}, Accuracy: {model_accuracy_loc}")

2022-03-10 03:00:02.160294: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-10 03:00:02.248086: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


Loss: 0.07991056889295578, Accuracy: 0.9800597429275513


In [25]:
predictions_test_loc = locmodel.predict(X_test_loc)
predictions_train_loc = locmodel.predict(X_train_loc)

2022-03-10 03:00:19.388858: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-10 03:00:19.423712: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [26]:
class_test_loc = predictions_test_loc > 0.25
f1_score_test_loc = sklearn.metrics.f1_score(y_test_loc, class_test_loc)
print(f"F1 Score: {f1_score_test_loc}")

F1 Score: 0.6390111041273832


In [27]:
class_train_loc = predictions_train_loc > 0.25
f1_score_train_loc = sklearn.metrics.f1_score(y_train_loc, class_train_loc)
print(f"F1 Score: {f1_score_train_loc}")

F1 Score: 0.6324833196234346


In [28]:
tn, fp, fn, tp = sklearn.metrics.confusion_matrix(y_test_loc, class_test_loc).ravel()
(tn, fp, fn, tp)

(80452, 508, 1215, 1525)

In [29]:
# Exporting location model
locmodel.save('locmodel')

INFO:tensorflow:Assets written to: locmodel/assets


INFO:tensorflow:Assets written to: locmodel/assets


## Spot checking results

In [30]:
loaded_signalmodel = load_model('signalmodel')

In [31]:
loaded_locmodel = load_model('locmodel')

In [32]:
slice_df = sig_df.iloc[29]

In [33]:
sigfeatures = []
sigtargets = []
locfeatures = []
loctargets = []

# x = slice_df["tr_x"]
# y = slice_df["tr_y"]
# z = slice_df["tr_z"]
tr = slice_df["trace"]
mag = abs(tr)
# mag = slice_df["magnitude"]
signal = slice_df["signal"]
p_arrival = slice_df["p_arrival"]
#d = {"x": x, "y":y, "z":z, "magnitude":mag}
d = {"trace": tr, "magnitude":mag}
temp_df = pd.DataFrame(data = d)

In [34]:
temp_df["STA"] = temp_df["magnitude"].rolling(roll_short).mean()
temp_df["LTA"] = temp_df["magnitude"].rolling(roll_long).mean()
temp_df["RAV"] = temp_df["STA"]/temp_df["LTA"]
temp_df["STV"] = temp_df["magnitude"].rolling(roll_short).var()
temp_df["LTV"] = temp_df["magnitude"].rolling(roll_long).var()
    
temp_df.dropna(inplace = True)
sigfeatures.append(temp_df.values)
sigtargets.append(signal)

In [35]:
loaded_signalmodel.predict(np.array(sigfeatures))

2022-03-10 03:01:20.719455: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-10 03:01:20.766065: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


array([[ 0.9884533]], dtype=float32)

In [36]:
sigtargets

[1]

In [37]:
start_ind = 0
end_ind = start_ind + window_size
    
while end_ind < (1000 - roll_long):
    # xwindow = temp_df["x"].iloc[start_ind:end_ind]
    # ywindow = temp_df["y"].iloc[start_ind:end_ind]
    # zwindow = temp_df["z"].iloc[start_ind:end_ind]
    magwindow = temp_df["magnitude"].iloc[start_ind:end_ind]
    trwindow = temp_df["trace"].iloc[start_ind:end_ind]
    ravwindow = temp_df["RAV"].iloc[start_ind:end_ind]
    stvwindow = temp_df["STV"].iloc[start_ind:end_ind]
    ltvwindow = temp_df["LTV"].iloc[start_ind:end_ind]

    window_data = {"trace":trwindow, "magnitude": magwindow,
                    "RAV": ravwindow, "STV": stvwindow, "LTV": ltvwindow}
        
    window_df = pd.DataFrame(data = window_data)

    locfeatures.append(window_df.values)

    if ((p_arrival-roll_long) >= start_ind) and ((p_arrival-roll_long) <= end_ind):
        loctargets.append(1)
    else:
        loctargets.append(0)

    start_ind += window_step
    end_ind = start_ind + window_size

In [38]:
loaded_locmodel.predict(np.array(locfeatures))

2022-03-10 03:01:23.093892: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-10 03:01:23.128385: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


array([[ 0.00321005],
       [ 0.00411626],
       [ 0.00354665],
       [ 0.01044728],
       [ 0.00430495],
       [ 0.00658699],
       [ 0.0050662 ],
       [ 0.00785536],
       [ 0.00833599],
       [ 0.00902405],
       [ 0.01379773],
       [ 0.00484164],
       [ 0.00517317],
       [ 0.00700498],
       [ 0.00300005],
       [ 0.0231615 ],
       [ 0.00489531],
       [ 0.00685779],
       [ 0.00239614],
       [ 0.00927619],
       [ 0.00294006],
       [ 0.0048273 ],
       [ 0.00748014],
       [ 0.00328153],
       [ 0.01095366],
       [ 0.00196791],
       [ 0.0112697 ],
       [ 0.00306058],
       [ 0.00566681],
       [ 0.4612412 ],
       [ 0.00176737]], dtype=float32)

In [39]:
loctargets

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0]

In [40]:
## Pseudo Coding Test Script
"""
1) Take in trace file
2) Normalize data
3) Calculate Fields for signalmodel
4) Predict whether or not the trace contains a signal

If YES:
5) Calculate Fields for locmodel
6) Predict where the pwave is
7) Take the max probability
8) Convert to a UTC timestamp
9) Return UTC timestamp

If NO:
5) Return None
"""

'\n1) Take in trace file\n2) Normalize data\n3) Calculate Fields for signalmodel\n4) Predict whether or not the trace contains a signal\n\nIf YES:\n5) Calculate Fields for locmodel\n6) Predict where the pwave is\n7) Take the max probability\n8) Convert to a UTC timestamp\n9) Return UTC timestamp\n\nIf NO:\n5) Return None\n'